# Semi-Analytic Model - Parameter Selection

## Initialization

In [ ]:
import os
# import warnings
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

# Silence annoying numpy errors
np.seterr(divide='ignore', invalid='ignore', over='ignore')
# warnings.filterwarnings("ignore", category=UserWarning)

# Plotting settings
mpl.rc('font', **{'family': 'serif', 'sans-serif': ['Times'], 'size': 15})
mpl.rc('lines', solid_capstyle='round')
mpl.rc('mathtext', fontset='cm')
mpl.style.use('default')   # avoid dark backgrounds from dark theme vscode
plt.rcParams.update({'grid.alpha': 0.5})


## Galaxy Stellar-Mass Function (GSMF)

### [Tomczak+2014](https://ui.adsabs.harvard.edu/abs/2014ApJ...783...85T/exportcitation)

- Includes single & double Schechter fits for various redshift bins, no fit to redshift evolution

In [ ]:
data = {
    0: {
        'z': [0.2, 0.5],
        'log10mstar': [11.05, 0.1],    # log10(M/Msol)
        'alpha': [-1.35, 0.04],
        'log10phistar': [-2.96, 0.10]  # log10(Phi / Mpc^-3 / dex)
    },
    1: {
        'z': [0.5, 0.75],
        'log10mstar': [11.00, 0.06],    # log10(M/Msol)
        'alpha': [-1.35, 0.04],
        'log10phistar': [-2.93, 0.07]  # log10(Phi / Mpc^-3 / dex)
    },
    2: {
        'z': [0.75, 1.00],
        'log10mstar': [11.16, 0.12],    # log10(M/Msol)
        'alpha': [-1.38, 0.04],
        'log10phistar': [-3.17, 0.11]  # log10(Phi / Mpc^-3 / dex)
    },
    3: {
        'z': [1.00, 1.25],
        'log10mstar': [11.09, 0.10],    # log10(M/Msol)
        'alpha': [-1.33, 0.05],
        'log10phistar': [-3.19, 0.11]  # log10(Phi / Mpc^-3 / dex)
    },
    4: {
        'z': [1.25, 1.50],
        'log10mstar': [10.88, 0.05],    # log10(M/Msol)
        'alpha': [-1.29, 0.05],
        'log10phistar': [-3.11, 0.08]  # log10(Phi / Mpc^-3 / dex)
    },
    5: {
        'z': [1.50, 2.00],
        'log10mstar': [10.97, 0.05],    # log10(M/Msol)
        'alpha': [-1.45, 0.05],
        'log10phistar': [-3.44, 0.08]  # log10(Phi / Mpc^-3 / dex)
    },
    6: {
        'z': [2.00, 2.50],
        'log10mstar': [11.28, 0.19],    # log10(M/Msol)
        'alpha': [-1.60, 0.08],
        'log10phistar': [-3.96, 0.19]  # log10(Phi / Mpc^-3 / dex)
    },
    7: {
        'z': [2.50, 3.00],
        'log10mstar': [11.35, 0.33],    # log10(M/Msol)
        'alpha': [-1.74, 0.12],
        'log10phistar': [-4.36, 0.29]  # log10(Phi / Mpc^-3 / dex)
    },    
}

In [ ]:
keys = list(data[0].keys())
print(f"{keys=}")
keys.pop(keys.index('z'))
fig, axes = plt.subplots(figsize=[10, 5], ncols=3)

for ii, vals in enumerate(data.values()):
    zbin = vals['z']
    zave = np.mean(zbin)
    zwid = np.diff(zbin)/2.0
    print(zave, zwid)
    for jj, (key, ax) in enumerate(zip(keys, axes)):
        val = vals[key]
        ax.errorbar(zave, val[0], xerr=zwid, yerr=val[1])
        if ii == 0:
            ax.set(xlabel='Redshift', title=key)
            ax.grid(True, alpha=0.25)

plt.show()